In [2]:
from sage.all import *
import numpy as np
import galois

In [3]:
def construct_hom_classical_code(num_bits: int, pc_mat: np.array) -> ChainComplex:
    """ 
    Construct chain complex for classical code from parity-check matrix

    Parameters:
    -----------
    num_bits - Number of bits of code
    pc_mat - parity-check matrix for code

    Returns:
    --------
    Chain complex representing code
    """
    # Define vector spaces for chain complex
    k_1 = len(pc_mat)
    C_1 = VectorSpace(GF(2), num_bits)
    C_0 = VectorSpace(GF(2), k_1)

    # Define boundary operators
    d_1 = matrix(pc_mat.tolist())

    # Define chain complex
    chain_complex_data = [C_1, d_1, C_0]
    code = ChainComplex(chain_complex_data, GF(2))
    return code

def construct_hom_CSS_code(num_qubits: int, pc_mat_X: np.array, pc_mat_Z: np.array) -> ChainComplex:
    """ 
    Construct chain complex for CSS code from parity-check matrices

    Parameters:
    -----------
    num_qubits - Number of qubits of code
    pc_mat_X - X parity-check matrix
    pc_mat_Z - Z parity-check matrix

    Returns:
    --------
    Chain complex representing CSS code
    """

    # Define vector spaces for chain complex
    k_1 = len(pc_mat_Z)
    k_2 = len(pc_mat_X)
    C_2 = VectorSpace(GF(2), k_1)
    C_1 = VectorSpace(GF(2), num_qubits)
    C_0 = VectorSpace(GF(2), k_2)

    # Define boundary operators
    d_2 = matrix((pc_mat_Z.T).tolist())
    d_1 = matrix(pc_mat_X.tolist())

    # Define chain complex
    chain_complex_data = [C_2, d_2, C_1, d_1, C_0]
    code = ChainComplex(chain_complex_data, GF(2))
    return code

In [20]:
# Let us construct the Steane Code
x_pc_mat = np.array([[0,0,0,1,1,1,1], [0,1,1,0,0,1,1,], [1,0,1,0,1,0,1]])
z_pc_mat = np.array([[0,0,0,1,1,1,1], [0,1,1,0,0,1,1,], [1,0,1,0,1,0,1]])

steane_code = construct_hom_CSS_code(7, x_pc_mat, z_pc_mat)
ascii_art(steane_code)

                                  [0 0 1]      
                                  [0 1 0]      
                                  [0 1 1]      
                                  [1 0 0]      
            [0 0 0 1 1 1 1]       [1 0 1]      
            [0 1 1 0 0 1 1]       [1 1 0]      
            [1 0 1 0 1 0 1]       [1 1 1]      
 0 <-- C_2 <---------------- C_1 <-------- C_0 <-- 0 

In [18]:
# Let us try to construct the Toric code via a product of chain complexes of classical codes
mat = np.array([
    [1,1,0],
    [0,1,1],
    [1,0,1]
])

rep_code_1 = construct_hom_classical_code(3, mat)
rep_code_2 = construct_hom_classical_code(3, mat)
toric_code = rep_code_1.tensor(rep_code_2)
orthogonality_check = toric_code.differential(1) * toric_code.differential(0)
toric_code.homology()

{0: Vector space of dimension 1 over Finite Field of size 2,
 1: Vector space of dimension 2 over Finite Field of size 2,
 2: Vector space of dimension 1 over Finite Field of size 2}